In [146]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

import numpy as np
from mpl_toolkits import mplot3d
import matplotlib
import matplotlib.pyplot as plt
import scipy.optimize as optimize
import Estimate

from LimitedCommitmentModel import LimitedCommitmentModelClass

# plot style
linestyles = ['-','--','-.',':',':']
markers = ['o','s','D','*','P']
linewidth = 2
font_size = 17
font = {'size':font_size}
matplotlib.rc('font', **font)

plt.rcParams.update({'figure.max_open_warning': 0,'text.usetex': False})
path = 'output/'

SAVE = False

# c++ settings
do_compile = True
threads = 7

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


# Solve the model

In [147]:
# compile c++ files
T = 10
specs = {  
    'model 1':{'latexname':'limited', 'par':{'kappa1': 0.5, 'kappa2': 0.0, 'T':T,'threads':threads,'bargaining':1}},

}




# solve different models
models = {}
for m,(name,spec) in enumerate(specs.items()):
    print(f'{name} loading...',end='')
    
    # setup model
    models[name] = LimitedCommitmentModelClass(name=name,par=spec['par'])
    models[name].spec = spec

    compile_now = True if do_compile & (m==0) else False
    models[name].link_to_cpp(force_compile=compile_now)
    
    print(' solving...')
    %time models[name].solve() 

model 1 loading... solving...
CPU times: total: 37.4 s
Wall time: 8.89 s


In [148]:
#VED LAVT ASSET = SKILT? 
# LAVT ASSET -> Lavt forbrug , elasticitet mellem forbrug og labor supply, skal den være højere/lavere?
#Mange perioder, bliver skilti starten? Forudsiger overstående på et tidspunkt?
# Skilt --> arbejder mere, Det er som om det ikke er negativ nok at arbejde mere , disnytte ved at arbejde skal være højere!! Men arbejder folk så ikke for lidt? Prøv at teste med at ændre væri

model = models['model 1']
model.sim.init_A[:] =0.0 
print(model.par.kappa1)
model.simulate()
print('model1')
print(f'Couple {np.mean(model.sim.couple,0)}')
print(f'Laborw {np.mean(model.sim.labor_w,0)}')
print(f'Laborm {np.mean(model.sim.labor_m,0)}')
print(f'Consumption W {np.mean(model.sim.cons_w,0)}')
print(f'Consumption M {np.mean(model.sim.cons_m,0)}')
print(f'Assetw {np.nanmean(model.sim.Aw,0)}')
print(f'Assetm {np.nanmean(model.sim.Am,0)}')
#print(f'Asset {np.nanmean(model.sim.A,0)}')
print(f'HKw {np.mean(model.sim.Kw,0)}')
print(f'HKm {np.mean(model.sim.Km,0)}')

#model = models['model 2, limited']
#model.simulate()
#print('model2')
#print(f'Couple {np.mean(model.sim.couple,0)}')
#print(f'Laborw {np.mean(model.sim.labor_w,0)}')
#print(f'Laborm {np.mean(model.sim.labor_m,0)}')
#print(f'Consumption W {np.mean(model.sim.cons_w,0)}')
#print(f'Consumption M {np.mean(model.sim.cons_m,0)}')
#print(f'Asset {np.nanmean(model.sim.A,0)}')
#print(f'HKw {np.mean(model.sim.Kw,0)}')
#print(f'HKm {np.mean(model.sim.Km,0)}')


#model = models['model 4, limited']
#model.simulate()
#print('model3')
#print(f'Couple {np.mean(model.sim.couple,0)}')
#print(f'Laborw {np.mean(model.sim.labor_w,0)}')
#print(f'Laborm {np.mean(model.sim.labor_m,0)}')
#print(f'Consumption W {np.mean(model.sim.cons_w,0)}')
#print(f'Consumption M {np.mean(model.sim.cons_m,0)}')
#print(f'Asset {np.nanmean(model.sim.A,0)}')
#print(f'HKw {np.mean(model.sim.Kw,0)}')
#print(f'HKm {np.mean(model.sim.Km,0)}')

model = models['model 1']
#print(model.sim.cons_m)
#print(model.sol.cons_m_single[4])
print(np.nanmin(model.sim.cons_m))
print(np.nanmin(model.sim.Am))
print(np.nanmin(model.sim.Km))



print(np.nanmin(model.sim.labor_m))
print(np.nanmin(model.sim.labor_w))


print(np.nanmin(model.sol.labor_m_single))


print(np.nanmax(model.sim.Am))
print(np.nanmax(model.sim.Km))

#print(model.sol.labor_m_single)





0.5
model1
Couple [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Laborw [1.         1.         1.         1.         1.         1.
 0.99498953 0.99926338 0.99946083 0.57462344]
Laborm [1.       1.       1.       1.       1.       1.       1.       1.
 1.       0.493427]
Consumption W [   5.47394639    6.14086603   15.3240716    24.23082732   67.37525162
  113.19964708  267.03043303  175.26300527  515.35729948 1266.97868939]
Consumption M [1.21250758e-01 4.79228165e-01 1.06849091e+00 1.77355816e+01
 2.87526314e+01 3.64858310e+01 4.40337344e+01 5.33182830e+01
 6.33188616e+01 2.09933042e+03]
Assetw [9.99999999e-07 7.24451826e+00 7.50000000e+00 7.48909717e+00
 7.49741646e+00 7.20530350e+00 7.17891486e+00 7.46382225e+00
 7.37552733e+00 6.71470065e-01]
Assetm [5.35269663 7.5        7.5        7.5        7.5        7.5
 7.5        7.5        7.5        7.47935784]
HKw [0.         1.00025147 1.90031925 2.71094317 3.43934181 4.0935512
 4.68396328 5.21205773 5.68575559 6.11856954]
HKm [0.         1.00102557 1.

# Estimate the model

In [209]:
#Estimate the model one time
#model = models['model 3']

model.sim.init_A[:] = 6.0 # temp just to check that it works, if they have too little initial saving they will just consume everything, and saving is therefore close to zero, and the model cannot be estimated

np.random.seed(model.par.seed)
data = Estimate.create_data(model,start_p = 1, end_p = 4, to_xl = True)
data_reg = Estimate.aux_est(data,print_reg=True)
data_reg, Wald_FC, Wald_NC = Estimate.main_est(data_reg,print_reg=True) 

Residuals from hours equation
                                 OLS Regression Results                                
Dep. Variable:        delta_log_Labor   R-squared (uncentered):                   0.452
Model:                            OLS   Adj. R-squared (uncentered):              0.452
Method:                 Least Squares   F-statistic:                          2.697e+04
Date:                Mon, 22 Jan 2024   Prob (F-statistic):                        0.00
Time:                        14:54:42   Log-Likelihood:                      2.0564e+05
No. Observations:               98028   AIC:                                 -4.113e+05
Df Residuals:                   98025   BIC:                                 -4.112e+05
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                       coef    std err          t      P>|t|      [0.025      0.975]
-----

c:\Users\czk481\Anaconda3\lib\site-packages\statsmodels\base\model.py:1906: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


                                 OLS Regression Results                                
Dep. Variable:                   uhat   R-squared (uncentered):                   0.995
Model:                            OLS   Adj. R-squared (uncentered):              0.995
Method:                 Least Squares   F-statistic:                          1.289e+06
Date:                Mon, 22 Jan 2024   Prob (F-statistic):                        0.00
Time:                        14:54:45   Log-Likelihood:                      3.0638e+05
No. Observations:               65352   AIC:                                 -6.127e+05
Df Residuals:                   65341   BIC:                                 -6.126e+05
Df Model:                          11                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------